# MS-SSIM - Multiscale Structural Similarity Index

Use MS-SSIM  to measure the perceptually quality difference between two videos computed frame by frame. Differs from ssim in that this function considers multiple scales. See [1](https://ieeexplore.ieee.org/document/1292216).

 - [1] Z. Wang, E. P. Simoncelli and A. C. Bovik, "Multi-scale structural similarity for image quality assessment," IEEE Asilomar Conference Signals, Systems and Computers, Nov. 2003.

## Example using ffmpeg

libvmaf is integrated into FFmpeg, to create complex filters to calculate MS-SSIM. This is an example where the distorted video is upscaled to the resolution of the reference video (360p to 1080p). The 'bicubic' interpolation method is used, see more on [FFmpeg](https://www.ffmpeg.org/ffmpeg-scaler.html). 

In [19]:
! mkdir -p results/msssim && \
ffmpeg -r 25 -i videos/dist/vp9/bbb_640x360_CRF30.webm \
       -r 25 -i videos/ref/bbb_1920x1080.mp4 \
       -lavfi "[0:v]scale=1920:1080:flags=bicubic,setpts=PTS-STARTPTS[distorted]; \
            [1:v]setpts=PTS-STARTPTS[reference]; \
            [distorted][reference]libvmaf=log_fmt=json:log_path=results/msssim/bbb_msssim_scaled.json:model='path=/vmaf/model/vmaf_float_v0.6.1.json':ms_ssim=1" -f null -   

## Graphical representation.

In this section, we plot the MS-SSIM values obtained per frame of the result obtained in Example 1. The arithmetic mean of the obtained values is also shown. 

In [16]:
import pandas as pd
import json
import plotly.express as px

path_file='results/msssim/bbb_msssim_scaled.json'

with open(path_file) as f:
  data = json.load(f)

msssim_frames=[]

for frame in data['frames']:
    msssim_frames.append(frame['metrics']['float_ms_ssim'])
        
df=pd.DataFrame(msssim_frames, columns=['float_ms_ssim'])

fig = px.line(df,x=df.index, y='float_ms_ssim',
              title='MS-SSIM by frame',labels={"index": "Frame"}, template='simple_white', markers=True)


fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()